In [64]:
import os
import pymol
from pymol import cmd
import numpy as np
import pandas as pd
import shutil


In [65]:
def myFunc(e):
    n = e[0:]
    print(n)
    n = n.replace('-out.mol2','')
    n = n.replace('A','')
    n = n.replace('-','')
    print(n)
    return int(n)

In [71]:
def make_conformers(path, csv_path):
    
    n_residue = path[6:]
    n_residue = n_residue.replace('_mol2','')
    n_residue = int(n_residue)
    file_list = os.listdir(path)
    file_list.sort(key = myFunc)
    df = pd.read_csv(csv_path)
    #define_numbers = list(df.iloc[:,0])
    
    #print(np.shape(define_numbers))
    for i in range(640):
        
        file_listFirstRingLength = ""
        file_listSecondRingLength = ""

        reachedFirstBreak = False
        Acounter = 0
        print(file_list[i])
        for c in file_list[i]:
            if(not c.isdigit()):
                if c == 'A':
                    Acounter += 1
                reachedFirstBreak = True
                
            if(c.isdigit() and not reachedFirstBreak):
                file_listFirstRingLength += c
                
            if(c.isdigit() and reachedFirstBreak and Acounter != 2 ):
                file_listSecondRingLength += c 
        
        file_listFirstRingLength  = int(file_listFirstRingLength)
        file_listSecondRingLength = int(file_listSecondRingLength) 
        
        print("new finder 1: " + str(file_listFirstRingLength))
        print("new finder 1: " + str(file_listSecondRingLength))

            
#         file_listFirstRingLength = int(file_list[i][0])
#         if file_listFirstRingLength == 1 and file_list[i][3] == '-':
#             file_listSecondRingLength = int(file_list[i][4]) 
#         else:
#             file_listSecondRingLength = int(file_list[i][3])

        
#         print(file_list[i][1].isdigit())

#         if file_list[i][1].isdigit():
#             file_listFirstRingLength = 10
        
#         #left off here, just trying to get if it is 10A to work
#         if file_list[i][4].isdigit() and file_listFirstRingLength != 10:
#             file_listSecondRingLength = 10
#         elif file_list[i][5].isdigit() and file_listFirstRingLength == 10:
#             file_listSecondRingLength = 10
            
        print(file_listFirstRingLength)
        print(file_listSecondRingLength)
        
        totalRingLength = file_listFirstRingLength + file_listSecondRingLength
        print(totalRingLength)
        
        scaffold_id = i
        new_path = path + '/' + file_list[i]
        #print(new_path)
        file = open(new_path, 'r')
        output = open('CLIPS_modified_specific/output.mol2','w')
        txt = file.readlines()
        key = '@<TRIPOS>MOLECULE\n'
        molecule_number = []
        
        for x in range(len(txt)):
            if txt[x] == key:
                molecule_number.append(x)
        if len(molecule_number) > 1:
            distance = molecule_number[1] - molecule_number[0]
        else: 
            distance = len(txt)
        for y in range(len(molecule_number)):
            #print(i)
            start = molecule_number[y]
            for k in range(start, start+distance):
                key_2 = '@<TRIPOS>BOND\n'
                if txt[k] == key_2:
                    end_number = k
            for k in range(start, start+distance):
                key_3 = '@<TRIPOS>SUBSTRUCTURE\n'
                if txt[k] == key_3:
                    modify_number = k
            #define_number = int(define_numbers[scaffold_id])
            iterate = True;
            hasC19 = False;
            hasN19 = False;
            
            unkCounter = 1;
            unkCounterN19 = 1;

            for j in range(start, start+distance):       
                    detector = txt[j][5:7].replace(' ','')
                    totalNumberOfAtoms = 57 + (totalRingLength * 10)
                    
                    if txt[j][:10] == '      4 N4':
                        data = txt[j].replace('N4','N ')
                        data = data[:57] + '1' + data[58:] 
                        output.write(data)
                    elif txt[j][:10] == '      1 C1':
                        data = txt[j].replace('C1','CA')
                        data = data[:57] + '0' + data[58:] 
                        output.write(data)
                    elif txt[j][:10] == '      3 O3':
                        data = txt[j].replace('O3','O ')
                        data = data[:57] + '0' + data[58:] 
                        output.write(data)
                    elif txt[j][:10] == '      2 C2':
                        data = txt[j].replace('C2','C ')
                        data = data[:57] + '0' + data[58:] 
                        output.write(data)
                    elif txt[j][:10] == '      5 C5':
                        data = txt[j].replace('C5','CA')
                        #data = data[:57] + '1' + data[58:] 
                        output.write(data)
    

                    elif(txt[j][8:9].isalpha() and txt[j][9:10].isnumeric()):
                        #print(txt[j][57:58] + " " +  txt[j][59:60])
                        #print( txt[j][9:10])
                        
                        atom_id = int(txt[j][4:7])
                        #6 to 17
                        if atom_id > 5 and atom_id < 18:
                            data = txt[j]
                            lastUNK = totalRingLength + 4
                            data = txt[j].replace('1 UNK',str(lastUNK) + ' UNK')
                            output.write(data)
                        #18
                        if atom_id > 17 and atom_id < 19:
                            data = txt[j].replace('C'+str(atom_id),'CA ')
                            #change amino acid number by using this
                            if (txt[j + 1][:11] == '     19 C19'):
                                data = data.replace('1 UNK',str(lastUNK - 1) + ' UNK')
                            if (txt[j + 1][:11] == '     19 N19'):
                                data = data.replace('1 UNK',str(lastUNK - file_listFirstRingLength - 2) + ' UNK')
                            output.write(data)
                            #print(data)       
                        #check if 19 is N or C. If C then not going down center of ring. N going down center
                        if(atom_id >18 and atom_id < 20):
                            if(txt[j][:11] == '     19 C19'):
                                data1 = txt[j].replace('C'+str(atom_id),'C  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - 1) + ' UNK')
                                hasC19 = True;
                                #print(data1)        
                                output.write(data1)
                            #checks if 19 is N 
                            if(txt[j][:11] == '     19 N19'):
                                data1 = txt[j].replace('N'+str(atom_id),'N  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - file_listFirstRingLength - 2) + ' UNK')
                                hasN19 = True;
                                #print(data1)        
                                output.write(data1)
                        #changing amide 20 to 21
                        if atom_id > 19 and atom_id < 22:
                            #print(txt[j][:11] == '     '+str(atom_id)+ ' C'+str(atom_id))
                            if(txt[j][:11] == '     '+str(atom_id)+ ' N'+str(atom_id)and hasC19 == True):
                                data1 = txt[j].replace('N'+str(atom_id),'N  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - 1) + ' UNK')
                                #print(data1)
                                output.write(data1)
                            if(txt[j][:11] == '     '+str(atom_id)+ ' O'+str(atom_id)and hasC19 == True):
                                data1 = txt[j].replace('O'+str(atom_id),'O  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - 1) + ' UNK')
                                #print(data1)
                                output.write(data1)
                            if(txt[j][:11] == '     '+str(atom_id)+ ' C'+str(atom_id)and hasN19 == True):
                                data1 = txt[j].replace('C'+str(atom_id),'C  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - file_listFirstRingLength - 3) + ' UNK')
                                #print(data1)
                                output.write(data1)
                            if(txt[j][:11] == '     '+str(atom_id)+ ' O'+str(atom_id)and hasN19 == True):
                                data1 = txt[j].replace('O'+str(atom_id),'O  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - file_listFirstRingLength - 3) + ' UNK')
                                #print(data1)
                                output.write(data1)
                        # in this case 7-2 it woulld be 22 to 55, and 55 to 60 for middle cys backbone. Does first ring
                        if atom_id > 21 and atom_id < file_listFirstRingLength * 5 + 20 + 5 + 1 :
                            if (atom_id )% 5 == 0 and hasC19 == True:
                                unkCounter += 1;
                            #print(atom_id)
                            #print(unkCounter)
                            if (atom_id - 2 )% 5 == 0 and hasC19 == True:
                                data1 = txt[j].replace('N'+str(atom_id),'N  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - unkCounter) + ' UNK')
                                output.write(data1)
                            if (atom_id - 3 )% 5 == 0 and hasC19 == True:
                                data1 = txt[j].replace('C'+str(atom_id),'C  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - unkCounter) + ' UNK')
                                output.write(data1)
                            if (atom_id - 4 )% 5 == 0 and hasC19 == True:
                                data1 = txt[j].replace('O'+str(atom_id),'O  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - unkCounter) + ' UNK')
                                output.write(data1)
                            if (atom_id )% 5 == 0 and hasC19 == True:
                                data1 = txt[j].replace('C'+str(atom_id),'CA ')
                                data1 = data1.replace('1 UNK',str(lastUNK - unkCounter) + ' UNK')
                                output.write(data1)
                            if (atom_id - 6 )% 5 == 0 and hasC19 == True:
                                data1 = txt[j].replace('C'+str(atom_id),'CB ')
                                data1 = data1.replace('1 UNK',str(lastUNK - unkCounter) + ' UNK')
                                #print(unkCounter)
                                output.write(data1)
                        # in this case 7-2 it woulld be 22 to 31, and NOT middle cys backbone. Does second ring
                        if atom_id > 21 and atom_id < file_listSecondRingLength * 5 + 20 + 2 :
                            if (atom_id ) % 5 == 0 and hasN19 == True:
                                unkCounterN19 += 1
                            #for N19    
                            if (atom_id - 2 )% 5 == 0 and hasN19 == True:
                                data1 = txt[j].replace('C'+str(atom_id),'CA ')
                                data1 = data1.replace('1 UNK',str(lastUNK - file_listFirstRingLength - 2 - unkCounterN19) + ' UNK')
                                output.write(data1)
                            if (atom_id - 3 )% 5 == 0 and hasN19 == True:
                                data1 = txt[j].replace('C'+str(atom_id),'CB ')
                                data1 = data1.replace('1 UNK',str(lastUNK - file_listFirstRingLength - 2 - unkCounterN19) + ' UNK')
                                output.write(data1)
                            if (atom_id - 4 )% 5 == 0 and hasN19 == True:
                                data1 = txt[j].replace('N'+str(atom_id),'N  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - file_listFirstRingLength - 2 - unkCounterN19) + ' UNK')
                                output.write(data1)
                            if (atom_id )% 5 == 0 and hasN19 == True:
                                data1 = txt[j].replace('C'+str(atom_id),'C  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - file_listFirstRingLength - 2 - unkCounterN19) + ' UNK')
                                output.write(data1)
                            if (atom_id - 6 )% 5 == 0 and hasN19 == True:
                                data1 = txt[j].replace('O'+str(atom_id),'O  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - file_listFirstRingLength - 2 - unkCounterN19) + ' UNK')
                                output.write(data1)
                        #7-2 n19 32 to 69. does the first 7 ring. does not include last cys and ending stuff does get its N
                        if atom_id > file_listSecondRingLength * 5 + 20 + 1 and atom_id < file_listSecondRingLength * 5 + 20 + 2 + 1 + file_listFirstRingLength * 5 + 2 :
                            if (atom_id - 4) % 5 == 0 and hasN19 == True:
                                unkCounterN19 += 1                            
                            #for N19    
                            if (atom_id - 2 )% 5 == 0 and hasN19 == True:
                                data1 = txt[j].replace('C'+str(atom_id),'C  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - file_listSecondRingLength - file_listFirstRingLength- 3 + unkCounterN19) + ' UNK')
                                output.write(data1)
                            if (atom_id - 3 )% 5 == 0 and hasN19 == True:
                                data1 = txt[j].replace('O'+str(atom_id),'O  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - file_listSecondRingLength - file_listFirstRingLength- 3 + unkCounterN19) + ' UNK')
                                output.write(data1)
                            if (atom_id - 4 )% 5 == 0 and hasN19 == True:
                                data1 = txt[j].replace('N'+str(atom_id),'N  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - file_listSecondRingLength - file_listFirstRingLength- 3 + unkCounterN19) + ' UNK')
                                output.write(data1)
                            if (atom_id )% 5 == 0 and hasN19 == True:
                                data1 = txt[j].replace('C'+str(atom_id),'CA ')
                                data1 = data1.replace('1 UNK',str(lastUNK - file_listSecondRingLength - file_listFirstRingLength- 3 + unkCounterN19) + ' UNK')
                                output.write(data1)
                            if (atom_id - 6 )% 5 == 0 and hasN19 == True:
                                data1 = txt[j].replace('C'+str(atom_id),'CB ')
                                data1 = data1.replace('1 UNK',str(lastUNK - file_listSecondRingLength- file_listFirstRingLength - 3 + unkCounterN19) + ' UNK')
                                output.write(data1)
                        #7-2 n19 70 to 73 just last cys
                        if atom_id > file_listSecondRingLength * 5 + 20 + 2 + file_listFirstRingLength * 5 + 2  and atom_id < file_listSecondRingLength * 5 + 20 + 2 + 1 + file_listFirstRingLength * 5 + 2 + 5:
                            if (atom_id - 2 )% 5 == 0 and hasN19 == True:
                                data1 = txt[j].replace('N'+str(atom_id),'N  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - file_listSecondRingLength- file_listFirstRingLength - 3 + unkCounterN19) + ' UNK')
                                output.write(data1)
                            if (atom_id - 3 )% 5 == 0 and hasN19 == True:
                                data1 = txt[j].replace('O'+str(atom_id),'O  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - file_listSecondRingLength- file_listFirstRingLength - 3 + unkCounterN19) + ' UNK')
                                output.write(data1)
                            if (atom_id )% 5 == 0 and hasN19 == True:
                                data1 = txt[j].replace('C'+str(atom_id),'CA ')
                                data1 = data1.replace('1 UNK',str(lastUNK - file_listSecondRingLength- file_listFirstRingLength - 3 + unkCounterN19) + ' UNK')
                                output.write(data1)
                            if (atom_id - 6 )% 5 == 0 and hasN19 == True:
                                data1 = txt[j].replace('C'+str(atom_id),'C  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - file_listSecondRingLength- file_listFirstRingLength - 3 + unkCounterN19) + ' UNK')
                                output.write(data1)    
#                       # CSC last for n19 7-2 it is 74 to ending of molecule includes all H
                        if atom_id >file_listSecondRingLength * 5 + 20 + 2  + file_listFirstRingLength * 5 + 2 + 5 -1 and atom_id < totalNumberOfAtoms + 1 and hasN19 == True:
                            #print("atom id : " + str(atom_id))
                            #print(file_listSecondRingLength * 5 + 20 + 2  + file_listFirstRingLength * 5 + 2 + 5)
                            #print(file_listSecondRingLength * 5 + 20 + 2  + file_listFirstRingLength * 5 + 2 + 5 + 2)
                            
                            if atom_id >= file_listSecondRingLength * 5 + 20 + 2  + file_listFirstRingLength * 5 + 2 + 5 and atom_id <= file_listSecondRingLength * 5 + 20 + 2  + file_listFirstRingLength * 5 + 2 + 5 + 2:
                                #print("worked")
                                data = txt[j]
                                data = data.replace('1 UNK',str(lastUNK) + ' UNK')
                                output.write(data) 
                            else:
                                data = txt[j]
                                output.write(data)
                        #just include CSC for n19 top one includes everything after CSC
#                         if atom_id >file_listSecondRingLength * 5 + 20 + 2  + file_listFirstRingLength * 5 + 2 + 5 -1 and atom_id < file_listSecondRingLength * 5 + 20 + 2  + file_listFirstRingLength * 5 + 2 + 5 and hasN19 == True:
#                             data = txt[j]
#                             data = data.replace('1 UNK',str(lastUNK) + ' UNK')
#                             output.write(data) 
                        #n19 the rest after CSC change stuff here if H UNK needs to change
#                         if atom_id >file_listSecondRingLength * 5 + 20 + 2  + file_listFirstRingLength * 5 + 3 + 7 and atom_id < totalNumberOfAtoms  and hasN19 == True:
#                             data = txt[j]
#                             output.write(data) 
                        #for 7-2 it would be 61 to 63 for the C S elements for c 19
                        if atom_id > file_listFirstRingLength * 5 + 20 + 5 and atom_id < file_listFirstRingLength * 5 + 20 + 5 + 3 + 1 and hasC19 == True:
                            data = txt[j]
                            data = data.replace('1 UNK',str(lastUNK) + ' UNK')
                            output.write(data)
                        #for 7-2 64 to 74 76 just last C=O for c19
                        if atom_id > file_listFirstRingLength * 5 + 20 + 5 + 3  and atom_id < file_listFirstRingLength * 5 + 20 + 5 + 3 + file_listSecondRingLength * 5 + 3 + 1:
                            if (atom_id - 5 )% 5 == 0 and hasC19 == True:
                                unkCounter += 1;
                            if (atom_id - 4 )% 5 == 0 and hasC19 == True:
                                data1 = txt[j].replace('N'+str(atom_id),'N  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - unkCounter) + ' UNK')
                                output.write(data1)
                            if (atom_id - 5 )% 5 == 0 and hasC19 == True:
                                data1 = txt[j].replace('C'+str(atom_id),'C  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - unkCounter) + ' UNK')
                                output.write(data1)
                            if (atom_id - 6 )% 5 == 0 and hasC19 == True:
                                data1 = txt[j].replace('O'+str(atom_id),'O  ')
                                data1 = data1.replace('1 UNK',str(lastUNK - unkCounter) + ' UNK')
                                output.write(data1)
                            if (atom_id -7)% 5 == 0 and hasC19 == True:
                                data1 = txt[j].replace('C'+str(atom_id),'CA ')
                                data1 = data1.replace('1 UNK',str(lastUNK - unkCounter) + ' UNK')
                                output.write(data1)
                            if (atom_id - 8 )% 5 == 0 and hasC19 == True:
                                data1 = txt[j].replace('C'+str(atom_id),'CB ')
                                data1 = data1.replace('1 UNK',str(lastUNK - unkCounter) + ' UNK')
                                output.write(data1)   
                        #for c19 last ccs       
                        if atom_id > file_listFirstRingLength * 5 + 20 + 5 + 3 + file_listSecondRingLength * 5 + 3 and atom_id < totalNumberOfAtoms + 1 and hasC19 == True:
                            data = txt[j]
                            output.write(data)
                    else:
                        output.write(txt[j])
                        #print(file_listFirstRingLength * 5 + 20 + 5 + 3 + file_listSecondRingLength * 5 + 3)



                        #print(atom_id)
                    
                    #elif iterate == False:
                     #   output.write(txt[j])

#                     elif(j>=8 and txt[j][4:7].replace(' ','').isnumeric() and j<155):
                        
#                         id = txt[j].replace('C' + str(6),'C ')
                        
#                         output.write(id)
#                     print(txt[j][:7])
                    


                        
#                     if txt[j][:11] == '     ' + str(define_number) + ' N' + str(define_number):
#                         data = txt[j].replace('N' + str(define_number),'N  ')
#                         data = data[:57] + str(n_residue + 2) + data[58:] 
#                         output.write(data)
#                     if txt[j][:11] == '     ' + str(define_number-2) + ' O' + str(define_number-2):
#                         data = txt[j].replace('O' + str(define_number-2),'O  ')
#                         data = data[:57] + str(n_residue + 2) + data[58:] 
#                         output.write(data)
#                     if txt[j][:11] == '     ' + str(define_number-3) + ' C' + str(define_number-3):
#                         data = txt[j].replace('C' + str(define_number-3),'C  ')
#                         data = data[:57] + str(n_residue + 2) + data[58:] 
#                         output.write(data)
#                     if txt[j][:11] == '     ' + str(define_number-4) + ' C' + str(define_number-4):
#                         data = txt[j].replace('C' + str(define_number-4),'CA ')
#                         data = data[:57] + str(n_residue + 2) + data[58:] 
#                         output.write(data)

#                     if detector.isnumeric():
#                         atom_id = int(txt[j][5:7])
#                         if atom_id > define_number and j < end_number:
#                             if 'C' in txt[j] and (atom_id-define_number)%5 == 1:
#                                 data = txt[j].replace(txt[j][8:11],'C  ')
#                                 residue_id = (atom_id - define_number-1)//5
#                                 residue_id = n_residue - int(residue_id) + 1
#                                 data = data[:57] + str(residue_id) + data[58:] 
#                                 output.write(data)
#                             elif 'O' in txt[j] and (atom_id-define_number)%5 == 2:
#                                 data = txt[j].replace(txt[j][8:11],'O  ')
#                                 residue_id = (atom_id - define_number-1)//5
#                                 residue_id = n_residue - int(residue_id) + 1
#                                 data = data[:57] + str(residue_id) + data[58:] 
#                                 output.write(data)
#                             elif 'C' in txt[j] and (atom_id-define_number)%5 == 3:
#                                 data = txt[j].replace(txt[j][8:11],'CA ')
#                                 residue_id = (atom_id - define_number-1)//5
#                                 residue_id = n_residue - int(residue_id) + 1
#                                 data = data[:57] + str(residue_id) + data[58:] 
#                                 output.write(data)
#                             elif 'C' in txt[j] and (atom_id-define_number)%5 == 4:
#                                 data = txt[j].replace(txt[j][8:11],'CB ')
#                                 residue_id = (atom_id - define_number-1)//5
#                                 residue_id = n_residue - int(residue_id) + 1
#                                 data = data[:57] + str(residue_id) + data[58:] 
#                                 output.write(data)
#                             elif txt[j][8] == 'N' and (atom_id-define_number)%5 == 0:
#                                 data = txt[j].replace(txt[j][8:11],'N  ')
#                                 residue_id = (atom_id - define_number-1)//5
#                                 residue_id = n_residue - int(residue_id) + 1
#                                 data = data[:57] + str(residue_id) + data[58:] 
#                                 output.write(data)
#                             else:
#                                 output.write(txt[j])
#                         elif atom_id <= define_number and j < end_number:
#                             if 'O' in txt[j]:
#                                 #print(txt[j], txt[j][8:11])
#                                 data_ = txt[j].replace(txt[j][8:11],'N  ')
#                                 #print(data_)
#                                 residue_id = int(n_residue) + 3
#                                 data = data_[:57] + str(residue_id) + data_[58:]
#                                 #print(data)
#                                 output.write(data)
#                             else:
#                                 residue_id = int(n_residue) + 3
#                                 data = txt[j][:57] + str(residue_id) + txt[j][58:]
#                                 output.write(data)
#                         elif j == modify_number + 1:
#                             data = txt[j][:39] + 'UNK' + txt[j][40:]
#                             output.write(data)
#                         else:
#                             output.write(txt[j])
#                     else:
#                         output.write(txt[j])
        print("i:" + str(i))
        output.close()
        dire_path = 'CLIPS_modified_specific/' + str(n_residue)
        if os.path.isdir(dire_path) == False:
            os.mkdir(dire_path)
        folder = file_list[scaffold_id][0:]
        folder = folder.replace('-out.mol2', '')
        dire_path = dire_path + '/' + folder
        if os.path.isdir(dire_path) == False:
            os.mkdir(dire_path)
        file =open('CLIPS_modified_specific/output.mol2', 'r')
        txt = file.readlines()
        data = file_list[scaffold_id][0:]
        data = data.replace('-out.mol2','_')
        for i in range(len(molecule_number)):
            start = molecule_number[i]
            name = data + str(i) + '.mol2'
            subfile = open(dire_path + '/' + name, 'w')
            for j in range(start, start+distance):
                subfile.write(txt[j])
            subfile.close()

In [72]:
make_conformers('CLIPS_2_mol2','CLIPS_csv/4.csv')

5A-3A1-out.mol2
531
5A-3A2-out.mol2
532
new finder 1: 5
new finder 1: 3
5
3
8
i:0
new finder 1: 5
new finder 1: 3
5
3
8
i:1


IndexError: list index out of range

In [ ]:
shutil.rmtree('schrodinger/')